In [54]:
import numpy as np
import pandas as pd
from datetime import datetime, date
import pytz

In [32]:
df = pd.read_csv("../input/logistics-shopee-code-league/delivery_orders_march.csv")

In [44]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [43]:
for col in ['pick', '1st_deliver_attempt', '2nd_deliver_attempt']:
    df[col] = df.apply(lambda row: datetime.fromtimestamp(row[col], tz=pytz.timezone('Asia/Singapore')).date() if not np.isnan(row[col]) else np.nan, axis = 1)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   orderid              int64 
 1   pick                 object
 2   1st_deliver_attempt  object
 3   2nd_deliver_attempt  object
 4   buyeraddress         object
 5   selleraddress        object
dtypes: int64(1), object(5)
memory usage: 145.4+ MB


In [61]:
hols = [date(2020, 3, i) for i in [8, 15, 22, 25, 29, 30, 31]] + [date(2020, 4, i) for i in [5]]

In [62]:
hols

[datetime.date(2020, 3, 8),
 datetime.date(2020, 3, 15),
 datetime.date(2020, 3, 22),
 datetime.date(2020, 3, 25),
 datetime.date(2020, 3, 29),
 datetime.date(2020, 3, 30),
 datetime.date(2020, 3, 31),
 datetime.date(2020, 4, 5)]

In [146]:
def giveRegion(s):
    ss = s.split()[-1].lower()
    if 'manila' in ss:
        return 'MM'
    if 'luzon' in ss:
        return 'LU'
    if 'visayas' in ss:
        return 'VI'
    if 'mindanao' in ss:
        return 'MI'
    return 'ER'

In [148]:
df['buyerregion'] = df.apply(lambda row: giveRegion(row['buyeraddress']), axis = 1)

In [149]:
df['sellerregion'] = df.apply(lambda row: giveRegion(row['selleraddress']), axis = 1)

In [89]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buyerregion,sellerregion
0,2215676524,2020-03-02,2020-03-05,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,MM,MM
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",MM,MM
2,2220979489,2020-03-04,2020-03-06,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",MM,MM
3,2221066352,2020-03-05,2020-03-07,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",MM,MM
4,2222478803,2020-03-04,2020-03-06,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,LU,MM


In [90]:
dct = {}
dct[('MM', 'MM')] = 3
dct[('MM', 'LU')] = 5
dct[('MM', 'VI')] = 7
dct[('MM', 'MI')] = 7
dct[('LU', 'MM')] = 5
dct[('LU', 'LU')] = 5
dct[('LU', 'VI')] = 7
dct[('LU', 'MI')] = 7
dct[('VI', 'MM')] = 7
dct[('VI', 'LU')] = 7
dct[('VI', 'VI')] = 7
dct[('VI', 'MI')] = 7
dct[('MI', 'MM')] = 7
dct[('MI', 'LU')] = 7
dct[('MI', 'VI')] = 7
dct[('MI', 'MI')] = 7

In [152]:
df['max_time'] = df.apply(lambda row: dct[(row['sellerregion'], row['buyerregion'])], axis = 1)

In [105]:
def count_days(start, end):
    count = 0
    for hol in hols:
        if start <= hol <= end:
            count += 1
    return (end - start).days - count


In [107]:
count_days(date(2020,3,2), date(2020,3,9))

6

In [108]:
df.to_csv("cp1.csv")

In [153]:
df['num_days_1'] = df.apply(lambda row: count_days(row['pick'], row['1st_deliver_attempt']), axis = 1)

In [154]:
df['state'] = df['num_days_1'] > df['max_time']

In [114]:
sum(df['state'])

441397

In [127]:
def count_days_v2(start, end, state):
    if state:
        return True
    elif type(end) == float:
        return False
    else:
        if count_days(start, end) > 3:
            return True
        else:
            return False

In [155]:
df['final_state'] = df.apply(lambda row: count_days_v2(row['1st_deliver_attempt'], row['2nd_deliver_attempt'], row['state']), axis = 1)

In [124]:
type(df['2nd_deliver_attempt'][0]) == float

True

In [156]:
subm_df2 = df[['orderid', 'final_state']]

In [160]:
sum(subm_df2['final_state'] != subm_df['final_state'])

3

In [159]:
subm_df2['final_state'] = subm_df2['final_state'].astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [133]:
subm_df

,orderid,final_state
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
...,...,...
3176308,31504087640510,0
3176309,31504147352227,0
3176310,31504462290482,0
3176311,31504851495943,0


In [134]:
sum(subm_df['final_state'])

762425

In [135]:
subm_df[subm_df['orderid'] == 1955598428]

,orderid,final_state


In [161]:
subm_df2.to_csv("submit3.csv", header = True, index = False)